## Lab 7: Eel Distribution Modeling with XGBoost

**Reference Paper:** [Elith et al. (2008)](https://ucsb.box.com/s/6k7636wsbogdg3orarxrlowke0ounbic)

In this lab, you will model the distribution of the eel species *Anguilla australis* using **boosted classification trees (BCTs)**, a machine learning technique that improves predictive performance by combining multiple decision trees. Elith et al. (2008) offered an early implementation of BRTs in an ecological setting to understand how environmental variables influence eel distribution.

You will work with **two datasets**:
1. **Training Data** – Used to build and evaluate your XGBoost model.
2. **Evaluation Data** – Used to assess model performance on unseen data.

To achieve the following objectives:
- Train and fine-tune an **XGBoost** model for classification of species presence/absence data.
- Compare your model’s performance to the approach used by Elith et al.


**Wherever applicable in this lab, use a random state of 808.**

### Step 0: Load libraries and data


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from scipy.stats import uniform, randint

# Download the datasets
model_data = pd.read_csv("/courses/EDS232/Data/model.data.csv").drop(columns=['Site'])
eval_data = pd.read_csv("/courses/EDS232/Data/eval.data.csv")

### Step 1:Initial Data Preprocessing
Let's get started by preparing our data. `Angaus` will be our target variable(`y`), and all other variables will be our features (`X`). Then encode the categorical feature using `LabelEncoder()`. The final step will be a bit different this time.  We don't need to split off testing data for the final model evaluation; a separate set (`eval_data`) will be used as in Elith et al.  We do, however, need to split our data in order to do the early stopping process. When splitting your data into training and validation, use a test size of 0.2 and a random state of 808. 

In [2]:
# Define target variable and features
y = model_data["Angaus"] 
X = model_data.drop(columns=["Angaus"])

# Encode categorical variables
label_encoder = LabelEncoder()
for col in X.select_dtypes(include=['object']).columns:
    X[col] = label_encoder.fit_transform(X[col])

# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=808)

# Check data dimensions
print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")
print(f"Evaluation data shape: {eval_data.shape}")

Training data shape: (800, 12)
Validation data shape: (200, 12)
Evaluation data shape: (500, 13)


### Step 2: Determine best number of trees using early stopping
As a guard against overfitting while maximizing performance, we use **early stopping**. We start with a large number of trees and allow XGBoost to determine the optimal number by stopping training when the validation error no longer improves.

The choice of hyperparameter starting values is important in this process. We begin with:
- `n_estimators=1000` to ensure the model has enough capacity to learn meaningful patterns.
- `learning_rate=0.1` as a reasonable default that balances learning speed and performance.
- `eval_metric="logloss"` as the metric of performance to optimize.
- `early_stopping_rounds=50` to halt training if no improvement is seen for 50 rounds, preventing unnecessary computations.
- `random_state = 808`

We then `fit()` our specified baseline model, passing in the training sets as usual and specifying validation sets values for the `eval_set` parameter.

Finally, get and print the best number of trees from the fitted baseline model.


In [3]:
# Initialize the XGBoost classifier
xgb_baseline = xgb.XGBClassifier(
    n_estimators=1000,
    learning_rate=0.1,     
    eval_metric="logloss",    
    early_stopping_rounds=50, 
    random_state=808,
    use_label_encoder=False 
)

# Fit the model with early stopping
xgb_baseline.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)], 
    verbose=True
)

# Print the best number of trees
best_trees = xgb_baseline.best_iteration
print(f"Optimal number of trees: {best_trees}")

[0]	validation_0-logloss:0.51354
[1]	validation_0-logloss:0.48803
[2]	validation_0-logloss:0.46738
[3]	validation_0-logloss:0.45126
[4]	validation_0-logloss:0.43766
[5]	validation_0-logloss:0.42555
[6]	validation_0-logloss:0.41728
[7]	validation_0-logloss:0.40769
[8]	validation_0-logloss:0.40082
[9]	validation_0-logloss:0.39562
[10]	validation_0-logloss:0.39263
[11]	validation_0-logloss:0.38984
[12]	validation_0-logloss:0.38489
[13]	validation_0-logloss:0.38327
[14]	validation_0-logloss:0.37803
[15]	validation_0-logloss:0.37652
[16]	validation_0-logloss:0.37337
[17]	validation_0-logloss:0.37156
[18]	validation_0-logloss:0.36987
[19]	validation_0-logloss:0.36868
[20]	validation_0-logloss:0.36809
[21]	validation_0-logloss:0.36721
[22]	validation_0-logloss:0.36763
[23]	validation_0-logloss:0.36570
[24]	validation_0-logloss:0.36577
[25]	validation_0-logloss:0.36666
[26]	validation_0-logloss:0.36559
[27]	validation_0-logloss:0.36552
[28]	validation_0-logloss:0.36422
[29]	validation_0-loglos

### Step 3: Tune Learning Rate

The (`learning_rate` hyperparameter controls how much each tree contributes to improving the model's performance. A *higher* learning rate allows the model to learn quickly but risks missing the optimal solution and overfitting, while a *lower* learning rate makes learning slower but can improve generalization.

To find the optimal value, we'll use **randomized search cross-validation** (`RandomizedSearchCV`) to test different learning rates in the 0.01 to 0.3 range. Instead of testing every possible value, this method samples a set number of candidates (`n_iter`) from a defined parameter distribution.  In this case, sampling 20 candidates from a uniform distribution between `0.01` and `0.31`. Check out the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html) on `scipy.stats.uniform` to see how it differs from `random.uniform`. Be sure to use a random state of 808.

After using `RandomizedSearchCV`, fit your model. Print the best learning rate. 


In [8]:
# Define the parameter distribution for learning rate
param_dist = {
    "learning_rate": uniform(0.01, 0.3 - 0.01)
}

# Initialize the XGBoost classifier 
xgb_model = xgb.XGBClassifier(
    n_estimators=best_trees,
    eval_metric="logloss",
    random_state=808,
    use_label_encoder=False
)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=20,
    scoring="neg_log_loss",
    cv=5,       
    random_state=808,
    verbose=1,
    n_jobs=-1 
)

# Fit the model
random_search.fit(X_train, y_train)

# Print the best learning rate
best_learning_rate = random_search.best_params_["learning_rate"]
print(f"Optimal learning rate: {best_learning_rate}")

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Optimal learning rate: 0.12179548710477123


### Step 4: Tune Tree-Specific Parameters

Now that we've determined the best number of tree and learning rate, we need to tune the complexity of individual trees in our model. Initialize your model with the best number of trees and learning rate.Then, define a parameter dictionary that takes on the following values:  

- `max_depth`(Controls how deep each tree can grow.  Takes integer values): A random integer from 3 to 10 ( inclusive of 3 and 10)
- `min_child_weight`( Determines the minimum number of samples required in a leaf node. Takes integer values) : A random integer from 1 to 10 ( inclusive of 1 and 10)
- `gamma` (Defines the minimum loss reduction needed to make a further split in a tree. Can take on values from a continuous range):  A uniform distribution from 0.05 to 0.10 - once again remember to check the `scipy.stats.uniform()` documentation! 
- `random_state = 808`

To find the best combination, we again use `RandomizedSearchCV`, allowing us to efficiently sample hyperparameters and evaluate different configurations using cross-validation. After fitting the model, print the best parameters. 



In [24]:
# Define the parameter distribution for tree complexity
param_dist_tree = {
    "max_depth": randint(3, 11), 
    "min_child_weight": randint(1, 11), 
    "gamma": uniform(0.05, 0.10 - 0.05)
}

# Initialize the XGBoost classifier
xgb_tree_tuned = xgb.XGBClassifier(
    n_estimators=best_trees, 
    learning_rate=best_learning_rate,
    eval_metric="logloss",
    random_state=808,
    use_label_encoder=False
)

# Set up RandomizedSearchCV
random_search_tree = RandomizedSearchCV(
    estimator=xgb_tree_tuned,
    param_distributions=param_dist_tree,
    n_iter=20,  
    scoring="neg_log_loss",
    cv=5,   
    random_state=808,
    verbose=1,
    n_jobs=-1
)

# Fit the model
random_search_tree.fit(X_train, y_train)

# Print the best parameters
best_tree_params = random_search_tree.best_params_
print(f"Optimal tree-specific parameters: {best_tree_params}")

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Optimal tree-specific parameters: {'gamma': 0.07521543766819237, 'max_depth': 8, 'min_child_weight': 10}


### Step 5: Tune Stochastic Components

Now, we are finally ready to tune the stochastic components of the XGBoost model.  These parameters help prevent overfitting by reducing correlation between trees. Initialize your model with the best number of trees, best learning rate,and your optimized tree values (**Note**: you can use \**best_tree_parameters to unpack the the dictionary of optimzed tree values) .Then, define a parameter dictionary that takes on the following values:  

- `subsample` (Controls the fraction of training samples used for each boosting round) : A uniform distribution between .5 and .10 (remeber to check `scipy.stats.uniform()` documentation! )
- `colsample_bytree`(Specifies the fraction of features to consider when building each tree) : A uniform distribution between .5 and .10
- `random_state = 808`

We again use `RandomizedSearchCV` to find the best combination of these parameters. After fitting the model, print the best parameters. 


In [25]:
# Define the parameter distribution
param_dist_stochastic = {
    "subsample": uniform(0.5, 0.5), 
    "colsample_bytree": uniform(0.5, 0.5)
}

# Initialize the XGBoost classifier
xgb_stochastic_tuned = xgb.XGBClassifier(
    n_estimators=best_trees,
    learning_rate=best_learning_rate,
    eval_metric="logloss",
    random_state=808,
    use_label_encoder=False,
    **best_tree_params
)

# Set up RandomizedSearchCV
random_search_stochastic = RandomizedSearchCV(
    estimator=xgb_stochastic_tuned,
    param_distributions=param_dist_stochastic,
    n_iter=20,
    scoring="neg_log_loss",
    cv=5,
    random_state=808,
    verbose=1,
    n_jobs=-1 
)

# Fit the model
random_search_stochastic.fit(X_train, y_train)

# Print the best parameters
best_stochastic_params = random_search_stochastic.best_params_
print(f"Optimal stochastic parameters: {best_stochastic_params}")

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Optimal stochastic parameters: {'colsample_bytree': 0.9365653342296307, 'subsample': 0.8785363858814997}


### Step 6: Final Model Training and Evaluation

With the best hyperparameters selected, we now train the final model on the full training dataset and evaluate it on the separate evaluation dataset.

1. Prepare the evaluation data in the same manner as you did the training data

2. Train final model using the best parameters found in previous tuning steps (`best_tree_params`, `best_stochastic_params`).Set  `eval_metric = "logloss"` 

3. Fit the model to the full training dataset and predict on the evaluation data 


In [17]:
# Prepare the evaluation data
y_eval = eval_data["Angaus_obs"] 
X_eval = eval_data.drop(columns=["Angaus_obs"])

# Encode categorical variables
for col in X_eval.select_dtypes(include=['object']).columns:
    X_eval[col] = label_encoder.transform(X_eval[col])

# Initialize the final XGBoost model
final_xgb_model = xgb.XGBClassifier(
    n_estimators=best_trees,
    learning_rate=best_learning_rate,
    eval_metric="logloss",
    random_state=808,
    use_label_encoder=False,
    **best_tree_params,
    **best_stochastic_params
)

# Train the final model
final_xgb_model.fit(X_train, y_train)

Final Model ROC-AUC Score: 0.8680


### Step 7: Model Performance

Compute and print the AUC and feature importances for your model.

In [20]:
# Make predictions on the evaluation data
y_eval_preds = final_xgb_model.predict_proba(X_eval)[:, 1]  # Get probabilities

roc_auc = roc_auc_score(y_eval, y_eval_preds)
print(f"Final Model AUC Score: {roc_auc:.4f}")

Final Model AUC Score: 0.8680


### Step 8: The comparison
How does your model's performance compare to the of Elith et al. (See Tables 2 and 3)?  Is there another way to compare the models in addition to predictive performance?  Whose model wins in that regard?

The final models AUC is super similar to the cross validation model in the Elith paper,they only varied by .001 with Elith et al. having 0.869, and my model having the score of 0.868. So, that would imply our predictive powers are very similar. Additionally, my model performed slightly better than theres on the independent data (0.858). This seems somewhat odd considering that my model only used 29 trees, vs. theres that used 1050. I suppose this is why using different models can be helpful. I don't like this result, I don't feel like my model should preform better than theirs, it makes me uncomfortable. Perhaps it had to do with the tree depth. 